## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import string
import vtb_scorekit
from vtb_scorekit.data import DataSamples
from pytest import approx
import ipytest
import pytest

ipytest.autoconfig()

pd.set_option("display.float_format", lambda x: "%.5f" % x)

(CVXPY) Jul 18 01:29:00 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jul 18 01:29:00 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')


In [2]:
# проверка, что фреймворк нужной версии
assert vtb_scorekit.__version__ == "1.2.2"

Возможные кейсы:
* столбец с числами float64, которые не должны быть сжаты;
* столбец с числами float32, но представнленные float64, которые должны быть сжаты до float32;
* столбец с числами int64, которые не должны быть сжаты;
* столбец с числами int32, но представнленные int64, которые должны быть сжаты до int32;
* столбец с числами int16, но представнленные int64, которые должны быть сжаты до int16;
* столбец с числами int8,  но представнленные int64, которые должны быть сжаты до int8;
Дополнительно создаем столбцы, содержащие спец. значения
* Спец. значения 9999999, 999999999, -999999999,-9999999. При сжатии не должны потерять в точности.
    * в столбце с пропусками определяются как float64;
    * в столбце без пропусков определяются как int64;

## Собираем выборку для тестов

In [3]:
# дополнительный набор
additional_values = [
    [np.nan, np.nan, 0.0, 0.0, 65000.0],
    [
        np.nan,
        -999999999.0,
        999999999.0,
        -7777777.0,
        7777777.0,
    ],
    [
        np.nan,
        -999999999.0,
        999999999.0,
        -7777777.0,
        7777777.0,
    ],
    [0, 2, 3, 4, 5],
    [0, 22222, 3, 4, 5],
    [
        0,
        -999999999,
        999999999,
        -7777777,
        7777777,
    ],
    ["a", "A", "ZTR", "wer", "dfsa dfs"],
]

In [4]:
# словарь с пограничными значениями для генерации случайных чисел
limits_dict = {
    "f16_limit": 65000.0,
    "f32_limit": 12354360734952.1443214,
    "f64_limit": 440282346638528859811704183484516925440.0,
    "i8_limit": 120,
    "i16_limit": 32700,
    "i32_limit": 2147483600,
    "i64_limit": 9223372036854775000,
}

In [5]:
# фиксируем генерацию случайных чисел для воспроизводимости
np.random.seed(42)
# создаем набор случайных значений, дополнив выбранными
data_dict = {
    "to_float16": np.append(
        np.random.uniform(-limits_dict["f16_limit"], limits_dict["f16_limit"], [1, 20])[
            0
        ],
        additional_values[0],
    ),  # числа, которые будут сжаты в float16
    "to_float32": np.append(
        np.random.uniform(-limits_dict["f32_limit"], limits_dict["f32_limit"], [1, 20])[
            0
        ],
        additional_values[1],
    ),  # числа, которые будут сжаты в float32
    "to_float64": np.append(
        np.random.uniform(-limits_dict["f64_limit"], limits_dict["f64_limit"], [1, 20])[
            0
        ],
        additional_values[2],
    ),  # числа, которые не должны быть сжаты
    "to_int8": np.append(
        np.random.randint(-limits_dict["i8_limit"], limits_dict["i8_limit"], 20),
        additional_values[3],
    ),  # числа, которые будут сжаты в int8
    "to_int16": np.append(
        np.random.randint(-limits_dict["i16_limit"], limits_dict["i16_limit"], 20),
        additional_values[4],
    ),  # числа, которые будут сжаты в int16
    "to_int32": np.append(
        np.random.randint(-limits_dict["i32_limit"], limits_dict["i32_limit"], 20),
        additional_values[5],
    ),  # числа, которые будут сжаты в int32
    "to_int64": np.append(
        np.random.randint(-limits_dict["i64_limit"], limits_dict["i64_limit"], 20),
        additional_values[5],
    ),  # числа, которые не должны быть сжаты
    "to_category": np.append(
        np.random.choice(list(string.ascii_uppercase), 20), additional_values[6]
    ),
}

* Баг, что метод DataSamples перезаписывает подаваемый датафрейм.

In [6]:
original_df = pd.DataFrame(data_dict)
original_df

,to_float16,to_float32,to_float64,to_int8,to_int16,to_int32,to_int64,to_category
0,-16309.78455,2763742021297.80078,-332819785804362643784273142287706357760.00000,87,29892,1503404280,8701552265281353973,S
1,58592.85983,-8907645785339.20117,-4247042668517015323795297186101592064.00000,116,-30673,529561463,6436323927311700520,V
2,30159.21244,-5135839985372.35742,-410001029090454956392547676585782673408.00000,-39,-30005,-217107653,4090187672963768321,B
3,12825.60295,-3302027991807.59375,360433094308544611908576024096692764672.00000,-10,26139,-726287407,-4870218625151263194,J
4,-44717.57674,-1085454524149.75781,-212409831514667308241523277115145846784.00000,-68,15490,-1737700272,-4499745231403915634,M
5,-44720.71236,7046333399972.15234,143111385472850871466416075971613425664.00000,-97,-27442,-1874502561,-8477503916587494224,Y
6,-57449.13042,-7420676866758.88965,-165800578530795110455006623647183929344.00000,33,-10698,-554831322,3886044415066855222,U
7,47602.89895,351714774042.43555,17671190913116481358137087070081908736.00000,96,6804,-811824698,-7177797341225557679,F
8,13144.95153,2283445841773.49805,41131422802800695820968167417569083392.00000,67,459,725167725,-1119043923099072268,L
9,27049.43511,-11206630424891.62109,-277506039707795602440480052271374139392.00000,3,-18714,-750831865,-5502309463496081050,L


In [7]:
compressed_df = DataSamples(
    samples={
        "train": original_df.copy()
    },  # здесь copy из-за перезаписи подаваемого датафрейма
    compress=True,
    cat_columns=["to_category"],
).samples["train"]

[INFO] [2023-07-18 01:29:01] ---------------------------------------------------------------- Creating DataSamples ----------------------------------------------------------------
[INFO] [2023-07-18 01:29:01] Selected 8 features: ['to_float16', 'to_float32', 'to_float64', 'to_int8', 'to_int16', 'to_int32', 'to_int64', 'to_category']


## Тесты

In [24]:
%%ipytest -qq

# выборка после сжатия
@pytest.fixture(scope='module')
def get_compressed_df():
    ds = DataSamples(
        samples={"train": original_df.copy()},
        compress=True,
        cat_columns=['to_category']
    )
    return ds.samples['train']

@pytest.fixture(scope='module')
def get_original_df():
    return original_df.copy()

# Smoke test
def test_compress_data(get_original_df):
    DataSamples(
        samples={"train": get_original_df.copy()},
        compress=True,
        cat_columns=['to_category']
    )
    assert True

# Typing
def test_columns_dtypes(get_compressed_df):
    """
    Функция для тестирования типов данных в столбцах
    после сжатия. В названии столбца зашит предполагаемый
    тип данных после сжатия, например: to_float16 - это float16
    """
    # задаем датафрейм с названиями столбцов и типом данных
    columns_dtypes_df = get_compressed_df.copy().dtypes.astype('str').reset_index()
    # отбираем данные по условию, что
    # название столбца соответствует ожидаемому типу
    condition_dtypes = columns_dtypes_df.apply(lambda row: row['index'].split('_')[1] == row[0], 1)
    # не соответствующие типу названию
    # столбцы записываем в отдельную переменную
    not_match_columns = np.array2string(columns_dtypes_df[condition_dtypes == False]['index'].values, separator=',')
    msg = f'Столбцы, которые не соответствуют типу: {not_match_columns}'
    assert condition_dtypes.all(), msg

def test_int_and_cat_values_match(get_compressed_df, get_original_df, abs_tol=1e-4):
    """
    Функция для тестирования того, что после
    сжатия данные не теряют в точности
    Parameters:
    get_compressed_df : датафрейм со сжатыми данными
    get_original_df : исходный датафрейм
    abs_tol : абсолютная точность (по умолчания 0.0001)
    """
    df_compressed = get_compressed_df.copy()
    df_original = get_original_df.copy()

    selected_columns = ['to_int8', 'to_int16', 'to_int32', 'to_int64', 'to_category']

    for column in selected_columns:
        
        msg = f'Значения в столбце {column} не соответствуют'
        # если тип int или object, то сравниваем "в лоб"        
        if pd.api.types.is_integer_dtype(df_original[column]) \
        or df_original[column].dtypes in ['object', 'category']:
            if (df_original[column] == df_compressed[column]).all():
                continue
            else:
                assert False, msg  
        else:
            assert False, f'неизвестный тип в столбце{column}'
    
    assert True

def test_values_are_close(get_compressed_df, get_original_df, abs_tol=1e-4):
    """
    Функция для тестирования того, что после
    сжатия данные не теряют в точности
    Parameters:
    get_compressed_df : датафрейм со сжатыми данными
    get_original_df : исходный датафрейм
    abs_tol : абсолютная точность (по умолчания 0.0001)
    """
    df_compressed = get_compressed_df.copy()
    df_original = get_original_df.copy()

    for column in ['to_float16', 'to_float32', 'to_float64']:
        
        msg = f'Значения в столбце {column} не соответствуют'

        # если тип float, то сравниваем с заданной ранее точностью
        if pd.api.types.is_float_dtype(df_original[column]):

            if df_original[column].values == approx(
                df_compressed[column].values, abs=abs_tol, nan_ok=True
            ):
                continue
            
            else:
                assert False, msg
        
        else:
            assert False, f'неизвестный тип в столбце{column}'
    
    assert True

...F                                                                                         [100%]
============================================= FAILURES =============================================
______________________________________ test_values_are_close _______________________________________

get_compressed_df =      to_float16            to_float32  \
0  -16312.00000   2763741986816.00000   
1   58592.00000  -8907645779968.0000...99999         ZTR  
23    -7777777             -7777777         wer  
24     7777777              7777777    dfsa dfs  
get_original_df =      to_float16            to_float32  \
0  -16309.78455   2763742021297.80078   
1   58592.85983  -8907645785339.2011...99999         ZTR  
23    -7777777             -7777777         wer  
24     7777777              7777777    dfsa dfs  
abs_tol = 0.0001

    def test_values_are_close(get_compressed_df, get_original_df, abs_tol=1e-4):
        """
        Функция для тестирования того, что после
        сжатия да

## Анализ расхождений

In [28]:
def calc_abs_diff(original_sample: pd.Series, compressed_sample: pd.Series) -> None:
    """
    Функция рассчитаывает разницу между
    сравниваемыми выборками по модулю
    и выводит результат в виде датафрейма
    """
    print("----" + original_sample.name + "----")
    res_df = pd.merge(
        original_sample,
        compressed_sample,
        left_index=True,
        right_index=True,
        suffixes=("_original", "_compressed"),
    )
    res_df["diff"] = (res_df[res_df.columns[0]] - res_df[res_df.columns[1]]).abs()
    display(res_df)
    print("--end---")

In [29]:
[
    calc_abs_diff(original_df[col], compressed_df[col])
    for col in [x for x in original_df.columns if "float" in x]
]

----to_float16----


,to_float16_original,to_float16_compressed,diff
0,-16309.78455,-16312.00000,2.21545
1,58592.85983,58592.00000,0.85983
2,30159.21244,30160.00000,0.78756
3,12825.60295,12824.00000,1.60295
4,-44717.57674,-44704.00000,13.57674
5,-44720.71236,-44736.00000,15.28764
6,-57449.13042,-57440.00000,9.13042
7,47602.89895,47616.00000,13.10105
8,13144.95153,13144.00000,0.95153
9,27049.43511,27056.00000,6.56489


--end---
----to_float32----


,to_float32_original,to_float32_compressed,diff
0,2763742021297.80078,2763741986816.00000,34481.80078
1,-8907645785339.20117,-8907645779968.00000,5371.20117
2,-5135839985372.35742,-5135839789056.00000,196316.35742
3,-3302027991807.59375,-3302027952128.00000,39679.59375
4,-1085454524149.75781,-1085454548992.00000,24842.24219
5,7046333399972.15234,7046333202432.00000,197540.15234
6,-7420676866758.88965,-7420676931584.00000,64825.11035
7,351714774042.43555,351714770944.00000,3098.43555
8,2283445841773.49805,2283445944320.00000,102546.50195
9,-11206630424891.62109,-11206630834176.00000,409284.37891


--end---
----to_float64----


,to_float64_original,to_float64_compressed,diff
0,-332819785804362643784273142287706357760.00000,-332819785804362643784273142287706357760.00000,0.00000
1,-4247042668517015323795297186101592064.00000,-4247042668517015323795297186101592064.00000,0.00000
2,-410001029090454956392547676585782673408.00000,-410001029090454956392547676585782673408.00000,0.00000
3,360433094308544611908576024096692764672.00000,360433094308544611908576024096692764672.00000,0.00000
4,-212409831514667308241523277115145846784.00000,-212409831514667308241523277115145846784.00000,0.00000
5,143111385472850871466416075971613425664.00000,143111385472850871466416075971613425664.00000,0.00000
6,-165800578530795110455006623647183929344.00000,-165800578530795110455006623647183929344.00000,0.00000
7,17671190913116481358137087070081908736.00000,17671190913116481358137087070081908736.00000,0.00000
8,41131422802800695820968167417569083392.00000,41131422802800695820968167417569083392.00000,0.00000
9,-277506039707795602440480052271374139392.00000,-277506039707795602440480052271374139392.00000,0.00000


--end---


[None, None, None]

## Выводы

Можно заметить, что при сжатии чисел с плавающей запятой сильно страдает точность. Это связано с тем, что границы для определения типа сжатия подобраны по максимальным возможным значениям. Например, для float32 наибольшим возможным значением является `3.4*10**38`, но при этом такой тип без потерь может сохранить всего 7 десятичных знаков. 

Также было выявлено, что класс DataSamples перезаписывает исходный DataFrame, подаваемый на вход